In [3]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jul 21 21:17:48 2019

@author: Team 3 
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import argparse
#from resnet import ResNet18
import torch.nn.functional as F
CUDA_VISIBLE_DEVICES=(2)
# if we need to use the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# cfg
if 0:
    parser = argparse.ArgumentParser(description='PyTorch MNIST Training')
    parser.add_argument('--outf', default='./model/', help='folder to output images and model checkpoints') #ourput saving address
    parser.add_argument('--net', default='./model/Resnet18.pth', help="path to net (to continue training)")  #to continue training address
    args = parser.parse_args()


EPOCH = 10   
pre_epoch = 0  
BATCH_SIZE = 64     
LR = 0.1        

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)# output+input
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)



transform = transforms.Compose([
    transforms.RandomCrop(28, padding=4),  
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.49],
                             std=[1])])

trainset = torchvision.datasets.MNIST(root='mnist/', train=True, download=True, transform=transform) 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)   

testset = torchvision.datasets.MNIST(root='mnist/', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0)

# ResNet
net = ResNet18().cuda()


criterion = nn.CrossEntropyLoss()  
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) 

# train
for epoch in range(pre_epoch, EPOCH):
    print('\nEpoch: %d' % (epoch + 1))
    net.train()
    sum_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        # perpare the data
        length = len(trainloader)
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        # forward + backward
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #print loss and acc once per one batch
        sum_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += predicted.eq(labels.data).cpu().sum()
    print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
          % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
    print("Waiting Test!")
    with torch.no_grad():
        correct = 0
        total = 0
        for data in testloader:
            net.eval()
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
        print('The ACC is ：%.3f%%' % (100 * correct / total))
      
                    


Epoch: 1
[epoch:1, iter:938] Loss: 0.320 | Acc: 89.000% 
Waiting Test!
The ACC is ：94.000%

Epoch: 2
[epoch:2, iter:1876] Loss: 0.122 | Acc: 96.000% 
Waiting Test!
The ACC is ：95.000%

Epoch: 3
[epoch:3, iter:2814] Loss: 0.105 | Acc: 96.000% 
Waiting Test!
The ACC is ：95.000%

Epoch: 4
[epoch:4, iter:3752] Loss: 0.099 | Acc: 96.000% 
Waiting Test!
The ACC is ：96.000%

Epoch: 5
[epoch:5, iter:4690] Loss: 0.101 | Acc: 96.000% 
Waiting Test!
The ACC is ：96.000%

Epoch: 6
[epoch:6, iter:5628] Loss: 0.093 | Acc: 97.000% 
Waiting Test!
The ACC is ：96.000%

Epoch: 7
[epoch:7, iter:6566] Loss: 0.090 | Acc: 97.000% 
Waiting Test!
The ACC is ：96.000%

Epoch: 8
[epoch:8, iter:7504] Loss: 0.091 | Acc: 97.000% 
Waiting Test!
The ACC is ：96.000%

Epoch: 9
[epoch:9, iter:8442] Loss: 0.091 | Acc: 97.000% 
Waiting Test!
The ACC is ：95.000%

Epoch: 10
[epoch:10, iter:9380] Loss: 0.090 | Acc: 97.000% 
Waiting Test!
The ACC is ：95.000%
